# Part 1

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import re
import bs4
import os
import mysql.connector

In [2]:
# use batch request to get the results
url='http://numbersapi.com/0..99' #The target url in short version.
user_agent = {'User-agent': 'Mozilla/5.0'} #Set up the user_agent.
result = requests.get(url, user_agent)

In [3]:
# organize the output format
for i in range(0,100):
    num = str(i)
    number = num.zfill(3)
    print(number,'-',result.json()[num])

000 - 0 is the atomic number of the theoretical element tetraneutron.
001 - 1 is the number of Gods in monotheism.
002 - 2 is the lowest channel of television in the United States, Canada, Argentina and Mexico on which television signals are broadcast.
003 - 3 is the number of novels or films in a trilogy and the number of interconnected works of art in a triptych.
004 - 4 is the number of legs most furniture has.
005 - 5 is the number of basic "pillars" of Islam.
006 - 6 is the number of points on a Star of David.
007 - 7 is the number of seconds it takes "Superman: Escape from Krypton" roller coaster to go from 0 to 100 miles per hour.
008 - 8 is the number of planets in the Solar System.
009 - 9 is the number of innings in a regulation, non-tied game of baseball.
010 - 10 is the number of years in a decade.
011 - 11 is the number of players in an American football team on the field at one time during play.
012 - 12 is the number of pence in a shilling in the former British currency 

# Part 2
## a)
### Q:
How would you modify your search query to only include buy-it-now (non-auction) items? What is the GET request's variable name corresponding to buy-it-now searches? How would you modify your search query to include 100 items per search result page? What is the GET request's variable name corresponding to items per page searches? Include your answers as a comment in your code marking it as Part 2-a. (This section does not involve coding)
### A:
I would add '&LH_BIN=1' at the end of the search query to only include buy-it-now (non-auction) items. Here BIN represents 'Buy it now'.
variable name: LH_BIN
I would add '&_ipg=100' at the end of the search query to include 100 items per search result page.
variable name: _ipg


## b)

In [4]:
sponsored = []
non_sponsored = []

In [5]:
url = 'https://www.ebay.com/sch/i.html?_nkw=ipad+4&LH_BIN=1&rt=nc&_ipg=100'

In [5]:
# download links of sponsored and non-sponsored items and put in the lists
for i in range(1,11):
    time.sleep(10) #Hold 10 seconds before the next scrape.
    num=str(i)
    newurl=url+'&_pgn='+num
    response=requests.get(newurl,headers = user_agent)
    soup=bs4.BeautifulSoup(response.text)
    titles=soup.find_all('a',class_='s-item__link')
    for title in titles:
        span_text=title.find_all('span')[0].text 
        if re.search('S.*P.*O.*N.*S.*O.*R.*E.*D',span_text): 
            sponsored.append(title['href'])
        else:
            non_sponsored.append(title['href'])

In [6]:
len(non_sponsored)

1000

## c)

In [6]:
# Create folders in the working directory of this python script
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

createFolder('./sponsored/')
createFolder('./non-sponsored/')

In [7]:
# get working directory
dir = os.getcwd()
dir

'/Users/jiaxuewan/Desktop/Winter Quarter/422'

In [185]:
# write the downloaded html pages into the above folders
for i in range(len(sponsored)):
    time.sleep(2)
    s_url = sponsored[i]
    response = requests.get(s_url,headers = user_agent)
    soup = bs4.BeautifulSoup(response.text)
    name = '/Users/jiaxuewan/Desktop/Winter Quarter/422/sponsored/'+re.findall(r'(\d{12})',s_url)[0]+'.htm'
    file = open(name, "w")
    file.write(response.text) #Write the html requested into it.
    file.close()

In [187]:
for i in range(len(non_sponsored)):
    time.sleep(2)
    s_url = non_sponsored[i]
    response = requests.get(s_url,headers = user_agent)
    soup = bs4.BeautifulSoup(response.text)
    name = '/Users/jiaxuewan/Desktop/Winter Quarter/422/non-sponsored/'+re.findall(r'(\d{12})',s_url)[0]+'.htm'
    file = open(name, "w")
    file.write(response.text) #Write the html requested into it.
    file.close()

## d & e) 
seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In [10]:
# get the file names
s_files= os.listdir('/Users/jiaxuewan/Desktop/Winter Quarter/422/sponsored/')

In [11]:
s_files = [name for name in s_files if name != ".DS_Store"]

In [12]:
len(s_files) 
# number of files is less than the number of pages that we download because there are some duplicated item ids so that the
# downladed page is covered

123

In [13]:
ns_files= os.listdir('/Users/jiaxuewan/Desktop/Winter Quarter/422/non-sponsored/')

In [14]:
len(ns_files)

962

Connect to Mysql

In [20]:
sql = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True)

In [21]:
mycursor=sql.cursor()
mycursor.execute("CREATE DATABASE if not exists eBay;")

In [27]:
mydb = mysql.connector.connect(user='root',
                              host='localhost',database = 'eBay')
mycursor=mydb.cursor()

In [28]:
mycursor.execute("create table if not exists eBay_items(seller_name VARCHAR(50),seller_score int(6),item_price int(10),items_sold int(10),best_offer_available VARCHAR(255),title VARCHAR(255),returns_allowed VARCHAR(255),shipping_price int(10),item_condition VARCHAR(255),sponsored VARCHAR(255),shipping_computed BOOLEAN);")



### There can be some listings that doesn't have seller information

254497174318.htm
184156175553.htm
143519896926.htm
143521382846.htm
153815637821.htm
133321695789.htm
184140936976.htm
274244855328.htm

The above AISNs are sold out when I extract the information from the htm pages on my Desktop.


173477802227.htm
182734395720.htm
202829537895.htm
181812696508.htm
183192071315.htm

These has no descriptions about return allowes or No return.

In [29]:
# sponsored

pos = 0
for file in s_files:
    name = '/Users/jiaxuewan/Desktop/Winter Quarter/422/sponsored/'+file
    text=open(name,'r').read() #Read the text from the file.
    soup=bs4.BeautifulSoup(text) #Create a beautifulsoup object using the html we got.
    ##### title #####
    title = soup.find_all('h1',class_='it-ttl')[0].text
    title = title.strip('Details about  \xa0')
    ##### seller_name, seller_score#####
    seller_name = soup.find_all('div',class_='bdg-90')[0].find('span',class_='mbg-nw').text
    seller_score = int(soup.find_all('div',class_='bdg-90')[0].find('span',class_='mbg-l').find('a').text)
    ##### item_sold #####
    if len(soup.find_all('a',class_='vi-txt-underline')) != 0:
        items_sold = soup.find_all('a',class_='vi-txt-underline')[0].text
        items_sold = items_sold.strip(' sold')
        items_sold = items_sold.replace(',','')
    else:
        items_sold = None
        
    ##### item_price #####
    if len(soup.find_all('div',id='vi-mskumap-none')) != 0:
        item_price = soup.find_all('div',id='vi-mskumap-none')[0].find('span',id='prcIsum').text
    elif len(soup.find_all('div',class_='vi-vpo-now2 u-flL w29 vi-price')) != 0:
        item_price = soup.find_all('div',class_='vi-vpo-now2 u-flL w29 vi-price')[0].find('span',id='prcIsum').text
    else:
        item_price = soup.find_all('div',class_='vi-vpo-now u-flL w29 vi-price')[0].find('span',id='prcIsum').text
    item_price = int(re.findall(r'\d{1}.+\.\d{2}',item_price)[0].replace('.','').replace(',',''))
   
    ##### returns_accepted #####
    if len(soup.find_all('div',id='why2buy')) != 0:
        returns_accepted = soup.find_all('div',id='why2buy')[0].text
        if len(re.findall('Returns accepted',returns_accepted)) != 0:
            returns_allowed = 'Returns accepted'
        elif len(re.findall('No returns',returns_accepted)) != 0:
            returns_allowed = 'No returns'
        else:
            returns_allowed = None
    else:
        returns_allowed = None
        

    
    ##### item_condition #####
    item_condition = soup.find_all('div',class_='nonActPanel')[0].find('div',class_='u-flL condText').text
    
    ##### best_offer_available #####
    if len(soup.find_all('a',id='boBtn_btn')) != 0:
        best_offer_available = soup.find_all('a',id='boBtn_btn')[0].text.strip()
    else:
        best_offer_available = None

        
    ##### sponsored #####     
    sponsored = 'sponsored'   
    
    
    ##### shipping_price #####
    if len(soup.find_all('span',id='fshippingCost')) != 0:
        if soup.find_all('span',id='fshippingCost')[0].text.strip() == 'FREE':
            shipping_price = 0
            shipping_computed = False
    elif len(soup.find_all('div',class_='u-flL sh-col')) != 0:
        if soup.find_all('div',class_='u-flL sh-col')[0].find('span',id='fshippingCost') is not None:
            shipping_price = soup.find_all('div',class_='u-flL sh-col')[0].find('span',id='fshippingCost').text.strip('\n$')
            if shipping_price != 'Free':
                shipping_price = int(shipping_price.replace('.',''))
                shipping_computed = False
    elif len(soup.find_all('strong',class_='sh_gr_bld')) != 0:
        shipping_price = soup.find_all('strong',class_='sh_gr_bld')[0].text
        shipping_price = 0
        shipping_computed = False
    elif len(soup.find_all('span',id='fshippingCost')) != 0:
        shipping_price = soup.find_all('span',id='fshippingCost')[0].text.strip()
        shipping_price = 0
        shipping_computed = False

    else:
        shipping_price = None
        shipping_computed = True 
    
    sql = "INSERT INTO eBay_items VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (seller_name, seller_score, item_price, items_sold, best_offer_available, title, returns_allowed, shipping_price, item_condition, sponsored, shipping_computed)
    mycursor.execute(sql, val)

    mydb.commit()

In [30]:
# non-sponsored
pos = 0
for file in ns_files:
    name = '/Users/jiaxuewan/Desktop/Winter Quarter/422/non-sponsored/'+file
    text=open(name,'r').read() #Read the text from the file.
    soup=bs4.BeautifulSoup(text) #Create a beautifulsoup object using the html we got.
    ##### title #####
    title = soup.find_all('h1',class_='it-ttl')[0].text
    title = title.strip('Details about  \xa0')
    ##### item_sold #####
    if len(soup.find_all('a',class_='vi-txt-underline')) != 0:
        items_sold = soup.find_all('a',class_='vi-txt-underline')[0].text
        items_sold = items_sold.strip(' sold')
        items_sold = items_sold.replace(',','')
    else:
        items_sold = None
        
    ##### seller_name, seller_score#####
    if len(soup.find_all('div',class_='bdg-90')) != 0:
        seller_name = soup.find_all('div',class_='bdg-90')[0].find('span',class_='mbg-nw').text
        seller_score = int(soup.find_all('div',class_='bdg-90')[0].find('span',class_='mbg-l').find('a').text)
    else:
        seller_name = None
        seller_score = None
        

    ##### item_price#####
    if len(soup.find_all('div',id='vi-mskumap-none')) != 0:
        item_price = soup.find_all('div',id='vi-mskumap-none')[0].find('span',class_='notranslate').text
    elif len(soup.find_all('div',id='bb_bdp')) != 0:
        item_price = soup.find_all('div',id='bb_bdp')[0].find('span',class_='notranslate').text
    elif len(soup.find_all('div',class_='vi-vpo-now2 u-flL w29 vi-price')) != 0:
        item_price = soup.find_all('div',class_='vi-vpo-now2 u-flL w29 vi-price')[0].find('span',id='prcIsum').text
    elif len(soup.find_all('div',class_='vi-vpo-now u-flL w29 vi-price')) != 0:
        item_price = soup.find_all('div',class_='vi-vpo-now u-flL w29 vi-price')[0].find('span',id='prcIsum').text
    else:
        item_price = None
    if item_price != None:
        item_price = int(re.findall(r'\d{1}.+\.\d{2}',item_price)[0].replace('.','').replace(',',''))
    else:
        item_price = None    
        
    ##### returns_accepted #####
    if len(soup.find_all('div',id='why2buy')) != 0:
        returns_accepted = soup.find_all('div',id='why2buy')[0].text
        if len(re.findall('Returns accepted',returns_accepted)) != 0:
            returns_allowed = 'Returns accepted'
        elif len(re.findall('No returns',returns_accepted)) != 0:
            returns_allowed = 'No returns'
        else:
            returns_allowed = None
    else:
        returns_allowed = None
    ##### item_condition #####   
    if len(soup.find_all('div',class_='nonActPanel')) != 0:
        item_condition = soup.find_all('div',class_='nonActPanel')[0].find('div',class_='u-flL condText').text
    else:
        item_condition = None
        
    ##### best_offer_available ##### 
    if len(soup.find_all('a',id='boBtn_btn')) != 0:
        best_offer_available = soup.find_all('a',id='boBtn_btn')[0].text.strip()
    else:
        best_offer_available = None
        
        
    ##### sponsored #####     
    sponsored = 'non_sponsored'    
    
    ##### shipping_price #####
    if len(soup.find_all('span',id='fshippingCost')) != 0:
        if soup.find_all('span',id='fshippingCost')[0].text.strip() == 'Free':
            shipping_price = 0
            shipping_computed = False
    elif len(soup.find_all('div',class_='u-flL sh-col')) != 0:
        if soup.find_all('div',class_='u-flL sh-col')[0].find('span',id='fshippingCost') is not None:
            shipping_price = soup.find_all('div',class_='u-flL sh-col')[0].find('span',id='fshippingCost').text.strip('\n$')
            shipping_price = int(shipping_price.replace('.',''))
            shipping_computed = False
    elif len(soup.find_all('strong',class_='sh_gr_bld')) != 0:
        shipping_price = soup.find_all('strong',class_='sh_gr_bld')[0].text
        shipping_price = 0
        shipping_computed = False
    elif len(soup.find_all('span',id='fshippingCost')) != 0:
        shipping_price = soup.find_all('span',id='fshippingCost')[0].text.strip()
        shipping_price = 0
        shipping_computed = False

    else:
        shipping_price = None
        shipping_computed = True
     

    sql = "INSERT INTO eBay_items VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (seller_name, seller_score, item_price, items_sold, best_offer_available, title, returns_allowed, shipping_price, item_condition, sponsored, shipping_computed)
    mycursor.execute(sql, val)

    mydb.commit()
        
        

## f)
Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations. 

In [31]:
# Before we update the table, we need to set the update mode.
mycursor.execute("""SET SQL_SAFE_UPDATES = 0;""")
mydb.commit()

In [32]:
mycursor.execute("update eBay_items set returns_allowed = if(returns_allowed = 'Returns accepted', 1, 0) where seller_name is not null")
mycursor.execute("update eBay_items set best_offer_available = if(best_offer_available = 'Make Offer', 1, 0) where seller_name is not null")
mydb.commit()

In [33]:
mycursor.execute("select title, best_offer_available from eBay_items where seller_name is null;")
mycursor.fetchall()

[('Apple iPad 4th Gen. 16GB, Wi-Fi, 9.7in - Wh', None),
 ('Apple iPad Mini MD540LL/A (16GB, Wi-Fi Verizon 4G, Black)', None),
 ('Apple iPad mini 4 128GB, Wi-Fi, 7.9in - Gold Good Condition', None),
 ('Apple 12.9" iPad Pro (Late 2018, 256GB, Wi-Fi + 4G LTE, Space Gray) MTJ02LL/A',
  None),
 ('Pad Mini 2 Wifi+4G Cellular BUNDLE-SIM, FREE DATA FOR LIFE (200MB/MO), OTTERBOX',
  None),
 ('Apple iPad 4th Gen., 16GB, Wi-Fi, 9.7" - Black (MD510LL/A) Cosmetic I',
  None),
 ('Apple iPad 4th Gen. 16GB, Wi-Fi, 9.7in - Wh', None),
 ('Apple iPad 4rd Generation A1458 MD514LL/A 32GB White/ Lot of 2', None),
 ('WARR. 2021 NEW 4G Apple iPad 6th Gen 128GB Wi-Fi Cellular Unlocked 9.7in SILVER',
  None)]

In [34]:
mycursor.execute("""SELECT sponsored,item_condition,avg(seller_score) as mean_seller_score, min(seller_score) as min_seller_score, max(seller_score) as max_seller_score,
                    avg(item_price) as mean_item_price, min(item_price) as min_item_price, max(item_price) as max_item_price,
                    avg(items_sold) as mean_items_sold, min(items_sold) as min_items_sold, max(items_sold) as max_items_sold,
                    avg(returns_allowed) as mean_returns_allowed, min(returns_allowed) as min_returns_allowed, max(returns_allowed) as max_returns_allowed,
                    avg(shipping_price) as mean_shipping_price, min(shipping_price) as min_shipping_price, max(shipping_price) as max_shipping_price,
                    avg(best_offer_available) as mean_best_offer_available, min(best_offer_available) as min_best_offer_available, max(best_offer_available) as max_best_offer_available
                    from eBay_items 
                    where item_condition is not null
                    and seller_name is not null
                    and item_price is not null
                    and items_sold is not null
                    and returns_allowed is not null
                    and best_offer_available is not null
                    
                    group by sponsored,item_condition;""")

results = mycursor.fetchall()

In [35]:
summary_stats_num_bin = pd.DataFrame(results,columns = ['sponsored','item_condition',
                                                'mean_seller_score','min_seller_score','max_seller_score',
                                                'mean_item_price','min_item_price','max_item_price',
                                                'mean_items_sold','min_items_sold','max_items_sold',
                                                'mean_returns_allowed','min_returns_allowed','max_returns_allowed',
                                                'mean_shipping_price','min_shipping_price','max_shipping_price',
                                                'mean_best_offer_available','min_best_offer_available','max_best_offer_available'])



In [36]:
summary_stats_num_bin
# Most of the non-sponsored items have calculated shipping price, which is noted as 'Null', so here in order to do the 
# summary stats on the columns together, we keep the NaN for shipping prices summary columns.

,sponsored,item_condition,mean_seller_score,min_seller_score,max_seller_score,mean_item_price,min_item_price,max_item_price,mean_items_sold,min_items_sold,max_items_sold,mean_returns_allowed,min_returns_allowed,max_returns_allowed,mean_shipping_price,min_shipping_price,max_shipping_price,mean_best_offer_available,min_best_offer_available,max_best_offer_available
0,sponsored,Manufacturer refurbished,91973.1977,1019,434655,19047.0465,8985,94999,411.6744,1,18628,0.953488,0,1,0.0000,0.0,0.0,0.000000,0,0
1,sponsored,Seller refurbished,146851.0000,476,2261592,12876.5217,7000,22559,382.3043,3,2332,0.956522,0,1,0.0000,0.0,0.0,0.000000,0,0
2,sponsored,Used,27859.2000,10981,84683,14496.2000,4999,22999,32.4000,1,147,0.600000,0,1,0.0000,0.0,0.0,0.200000,0,1
3,sponsored,Open box,66480.5714,12383,84683,17215.0000,9995,30575,94.2857,1,437,1.000000,1,1,0.0000,0.0,0.0,0.000000,0,0
4,non_sponsored,Manufacturer refurbished,155060.9367,1019,434655,21977.1139,5799,96999,169.1772,1,1615,0.987342,0,1,0.0000,0.0,0.0,0.000000,0,0
5,non_sponsored,Open box,11791.0000,444,89830,16790.4464,8995,39995,137.2321,1,3291,0.964286,0,1,0.0000,0.0,0.0,0.017857,0,1
6,non_sponsored,Seller refurbished,158998.4206,63,2261612,17879.7619,7299,99999,140.5397,1,1812,0.960317,0,1,None,NaN,NaN,0.047619,0,1
7,non_sponsored,New,11366.5000,1,186889,55783.6750,14999,189999,4.2250,1,17,0.800000,0,1,None,NaN,NaN,0.050000,0,1
8,non_sponsored,Used,26677.5088,66,257946,17272.2982,4499,99999,111.9123,1,2070,0.877193,0,1,None,NaN,NaN,0.228070,0,1


In [37]:
mycursor.execute("SELECT sponsored, item_condition, COUNT(*) FROM eBay_items group by sponsored,item_condition;")
results2 = mycursor.fetchall()

In [38]:
summary_stats_categorical = pd.DataFrame(results2,columns = ['sponsored','item_condition','count_sponsored_condition'])

In [39]:
summary_stats_categorical

,sponsored,item_condition,count_sponsored_condition
0,sponsored,Manufacturer refurbished,87
1,sponsored,Seller refurbished,23
2,sponsored,Used,5
3,sponsored,Open box,7
4,sponsored,New,1
5,non_sponsored,Manufacturer refurbished,122
6,non_sponsored,Used,441
7,non_sponsored,Open box,91
8,non_sponsored,None,8
9,non_sponsored,Seller refurbished,175


In [40]:
summary = summary_stats_num_bin.merge(summary_stats_categorical,on = ['sponsored','item_condition'])

In [41]:
pd.set_option('display.max_columns', None)
summary

,sponsored,item_condition,mean_seller_score,min_seller_score,max_seller_score,mean_item_price,min_item_price,max_item_price,mean_items_sold,min_items_sold,max_items_sold,mean_returns_allowed,min_returns_allowed,max_returns_allowed,mean_shipping_price,min_shipping_price,max_shipping_price,mean_best_offer_available,min_best_offer_available,max_best_offer_available,count_sponsored_condition
0,sponsored,Manufacturer refurbished,91973.1977,1019,434655,19047.0465,8985,94999,411.6744,1,18628,0.953488,0,1,0.0000,0.0,0.0,0.000000,0,0,87
1,sponsored,Seller refurbished,146851.0000,476,2261592,12876.5217,7000,22559,382.3043,3,2332,0.956522,0,1,0.0000,0.0,0.0,0.000000,0,0,23
2,sponsored,Used,27859.2000,10981,84683,14496.2000,4999,22999,32.4000,1,147,0.600000,0,1,0.0000,0.0,0.0,0.200000,0,1,5
3,sponsored,Open box,66480.5714,12383,84683,17215.0000,9995,30575,94.2857,1,437,1.000000,1,1,0.0000,0.0,0.0,0.000000,0,0,7
4,non_sponsored,Manufacturer refurbished,155060.9367,1019,434655,21977.1139,5799,96999,169.1772,1,1615,0.987342,0,1,0.0000,0.0,0.0,0.000000,0,0,122
5,non_sponsored,Open box,11791.0000,444,89830,16790.4464,8995,39995,137.2321,1,3291,0.964286,0,1,0.0000,0.0,0.0,0.017857,0,1,91
6,non_sponsored,Seller refurbished,158998.4206,63,2261612,17879.7619,7299,99999,140.5397,1,1812,0.960317,0,1,None,NaN,NaN,0.047619,0,1,175
7,non_sponsored,New,11366.5000,1,186889,55783.6750,14999,189999,4.2250,1,17,0.800000,0,1,None,NaN,NaN,0.050000,0,1,125
8,non_sponsored,Used,26677.5088,66,257946,17272.2982,4499,99999,111.9123,1,2070,0.877193,0,1,None,NaN,NaN,0.228070,0,1,441


## g)

Difference between sponsored and non-sponsored items:

1. The seller score of sponsored items have higher minimum level than non-sponsored items. For sponsored items, the minimum seller score is at least over 400. However, for non-sponsored items, it can be 0.
2. The average item price of sponsored items is lower than non-sponsored items when the condition is the same. ('Used','New',etc.) 
3. The average number of items sold is larger for sponsored items than that of non-sponsored for refurbished ones.
4. Non-sponsored items are more likely to have best offer available.

According to above findings, minimum seller score, best offer option can be used to predict the sponsor/non-sponsor items.
